In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [3]:
df = pd.read_csv('Churn_Modelling.csv')

df = df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])

one_hot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = one_hot_encoder_geo.fit_transform(df[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

df = pd.concat([df.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = df.drop('Exited', axis=1)
y = df['Exited']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)
with open('one_hot_encoder_geo.pkl', 'wb') as f:
    pickle.dump(one_hot_encoder_geo, f)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [6]:
# define the model and try different parameters

def create_model(neurons=32, layer=1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_dim=(X_train.shape[1],)))
    
    for _ in range(layer - 1):
        model.add(Dense(neurons, activation='relu'))
        
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [10]:
# create a KerasClassifier
model = KerasClassifier(
    layer=1,
    neurons=32,
    build_fn=create_model,
    verbose=1
    )

C:\Users\hoang\AppData\Local\Temp\ipykernel_21432\3555989047.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(


In [11]:
# grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layer': [1, 2, 3],
    'batch_size': [10, 20],
    'epochs': [50, 100]
}

In [ ]:
# perform grid search
grid = GridSearchCV(
    estimator=model, 
    param_grid=param_grid, 
    n_jobs=-1, cv=3
)
grid_result = grid.fit(X_train, y_train)

# print the best parameters
print("Best: %f using %s" % (grid.best_score_, grid.best_params_)) 